In [ ]:
%%capture

import importlib

from tools.compare_tools import *
import matplotlib.pyplot as plt
import numpy as np
import cv2
import cnn
from skimage import measure
from tools.load_nc import load_nc_sat, load_nc_sinmod
from matplotlib.patches import Rectangle
import pandas as pd
from scipy.interpolate import griddata, interp2d
from scipy.signal import resample
from tools import grid_conversion
from scipy.spatial import cKDTree

# Prepare data for analysis

### Load NC data

In [ ]:
# Load satellite netcdf data
fPath = 'D:/master/data/compare/satellite/phys_465.nc'
lons_sat,lats_sat,sst_sat,ssl_sat,sal_sat,uvel_sat,vvel_sat =  load_nc_sat(fPath)
# Get SINMOD data
fPath = 'D:/master/data/compare/sinmod/samples_2017.06.01_nonoverlap.nc'
x_sinmod,y_sinmod,_,lons_sinmod,lats_sinmod,sst_sinmod,ssl_sinmod,sal_sinmod,uvel_sinmod,vvel_sinmod =  load_nc_sinmod(fPath)

### Convert satellite data to north-polar sterographic projection

In [ ]:
# Get a polar steregraphic projection with cartesian irregular curvilinear grid for satellite data 
x_sat_2d, y_sat_2d = grid_conversion.bl2xy(*np.meshgrid(lons_sat, lats_sat), FE=3254000, FN=2560000, xy_res=1, slon=58, SP=60)

### Interpolate satellite data to high enough resolution such that extracting closest neighbor from the satellite measurements based on queries from the selected SINMOD grid won't have duplicates.

In [ ]:
# Interpolate data and the curvilienar grid axes such that it has a higher resolution than SINMOD
sst_sat_it = interp2d_masked(sst_sat, lons_sat, lats_sat, 4, 'linear')
ssl_sat_it = interp2d_masked(ssl_sat, lons_sat, lats_sat, 4, 'linear')
sal_sat_it = interp2d_masked(sal_sat, lons_sat, lats_sat, 4, 'linear')
uvel_sat_it = interp2d_masked(uvel_sat, lons_sat, lats_sat, 4, 'linear')
vvel_sat_it = interp2d_masked(vvel_sat, lons_sat, lats_sat, 4, 'linear')
x_sat_it = interp2d_masked(x_sat_2d.T, lons_sat, lats_sat, 4, 'linear')
y_sat_it = interp2d_masked(y_sat_2d.T, lons_sat, lats_sat, 4, 'linear')

### Select a grid that will be our basis of analysis

In [ ]:
# Get SINMOD Grid
#x_bnds = [100000, 500000]
#y_bnds = [1650000, 1900000]
# Mellom Grønland, Svalbard og Norge
x_bnds = [1800000, 2150000]
y_bnds = [1600000, 1870000]

#
x_bnds = [2370000, 2500000]
y_bnds = [800000, 1000000]

x_bnds = [700000, 1200000]
y_bnds = [900000, 1200000]

x_idxs = np.where((x_sinmod >= x_bnds[0]) & (x_sinmod <= x_bnds[1]))[0]
y_idxs = np.where((y_sinmod >= y_bnds[0]) & (y_sinmod <= y_bnds[1]))[0]

x_sinmod = np.ma.array(x_sinmod[x_idxs])
y_sinmod = np.ma.array(y_sinmod[y_idxs])
lons_sinmod = np.ma.array(lons_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
lats_sinmod = np.ma.array(lats_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
sst_sinmod = np.ma.array(sst_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
ssl_sinmod = np.ma.array(ssl_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
sal_sinmod = np.ma.array(sal_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
uvel_sinmod = np.ma.array(uvel_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
vvel_sinmod = np.ma.array(vvel_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])


In [ ]:

#fPath = 'D:/master/data/compare/sinmod/samples_2016.09.10_nonoverlap.nc'
#x_sinmod,y_sinmod,lons_sinmod,lats_sinmod,sst_sinmod,ssl_sinmod,sal_sinmod,uvel_sinmod,vvel_sinmod =  load_nc_sinmod(fPath)

x_bnds = [700000, 1200000]
y_bnds = [900000, 1200000]

#x_bnds = [400000, 880000]
#y_bnds = [150000, 450000]

# Mellom Svalbard og Grønland, potensielt mye rart som slipper mellom der
#x_bnds = [2200000, 2800000]
#y_bnds = [1750000, 2350000]
#
#x_bnds = [800000, 2400000]
#y_bnds = [1000000, 1900000]

#x_bnds = [1000000, 2000000]
#y_bnds = [1000000, 2100000]

x_idxs = np.where((x_sinmod >= x_bnds[0]) & (x_sinmod <= x_bnds[1]))[0]
y_idxs = np.where((y_sinmod >= y_bnds[0]) & (y_sinmod <= y_bnds[1]))[0]

x = np.ma.array(x_sinmod[x_idxs])
y = np.ma.array(y_sinmod[y_idxs])
lons = np.ma.array(lons_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
lats = np.ma.array(lats_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
sst = np.ma.array(sst_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
ssl = np.ma.array(ssl_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
sal = np.ma.array(sal_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
uvel = np.ma.array(uvel_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])
vvel = np.ma.array(vvel_sinmod[x_idxs[0]:x_idxs[-1]+1, y_idxs[0]:y_idxs[-1]+1])

uvel, vvel = rotate_vector(uvel, vvel, lons, 58)

fig, ax = plt.subplots(1,1,figsize=(15,12))
ax.pcolormesh(x, y, ssl.T, cmap='rainbow')
ax.quiver(x,y,uvel.T,vvel.T,scale=8)


### Create k-d tree to create a satellite grid with same axes as the selected SINMOD grid

In [ ]:
# Create cKDTree object to represent the full flattened cartesian coordinates of the satellite data grid
tree = cKDTree(list(zip(x_sat_it.flatten(), y_sat_it.flatten())))

In [ ]:
# Find indices of the satellite data nearest to the SINMOD coordinates we use
x_sinmod_2d, y_sinmod_2d = np.meshgrid(x_sinmod, y_sinmod)
d, inds = tree.query(list(zip(x_sinmod_2d.ravel(), y_sinmod_2d.ravel())), k = 1)

# Extract satellite data by query indexes to get same grid axes and apply the same landmass mask
sinmod_mask = np.ma.getmask(ssl_sinmod)
sst_sat_samegrid = np.ma.masked_where(sinmod_mask, sst_sat_it.flatten()[inds].reshape(uvel_sinmod.T.shape).T)
ssl_sat_samegrid = np.ma.masked_where(sinmod_mask, ssl_sat_it.flatten()[inds].reshape(uvel_sinmod.T.shape).T)
sal_sat_samegrid = np.ma.masked_where(sinmod_mask, sal_sat_it.flatten()[inds].reshape(uvel_sinmod.T.shape).T)
uvel_sat_samegrid = np.ma.masked_where(sinmod_mask, uvel_sat_it.flatten()[inds].reshape(uvel_sinmod.T.shape).T)
vvel_sat_samegrid = np.ma.masked_where(sinmod_mask, vvel_sat_it.flatten()[inds].reshape(uvel_sinmod.T.shape).T)

In [ ]:
# Rotate velocities to fit the shifted pole stereographic projection
uvel_sinmod, vvel_sinmod = rotate_vector(uvel_sinmod, vvel_sinmod, lons_sinmod, 58)
uvel_sat_samegrid, vvel_sat_samegrid = rotate_vector(uvel_sat_samegrid, vvel_sat_samegrid, lons_sinmod, 58)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,6), dpi=150)
fig.subplots_adjust(wspace=0.01, hspace=0)

#import matplotlib.gridspec as gridspec
#gs = gridspec.GridSpec(1, 2)
#gs.update(wspace=0.02, hspace=0.05) # set the spacing between axes. 

ax[0].pcolormesh(x_sinmod, y_sinmod, ssl_sat_samegrid.T, cmap='rainbow')
ax[0].quiver(x_sinmod,y_sinmod,uvel_sat_samegrid.T,vvel_sat_samegrid.T,scale=4)
#ax[0].streamplot(x_sinmod,y_sinmod,uvel_sat_samegrid.T,vvel_sat_samegrid.T,density=6)
ax[0].axes.get_xaxis().set_visible(False)
ax[0].axes.get_yaxis().set_visible(False)
ax[0].set_title('test')

ax[1].pcolormesh(x_sinmod, y_sinmod, ssl_sinmod.T, cmap='rainbow')
ax[1].quiver(x_sinmod,y_sinmod,uvel_sinmod.T,vvel_sinmod.T,scale=4) 
#ax[1].streamplot(x_sinmod,y_sinmod,uvel_sinmod.T,vvel_sinmod.T,density=6) 
ax[1].axes.get_xaxis().set_visible(False)
ax[1].axes.get_yaxis().set_visible(False)
ax[1].set_title('test')

# Process satellite data

### Perform pyramid sliding window on grid and return use CNN model to predict rectangles

In [ ]:
%%capture
importlib.reload(cnn)
model_fpath = 'D:/master/models/best_model_975.h5'
cnn_win_sizes = [((int(6), int(5)), 2, 2),((int(8), int(6)), 3, 2)]#,((int(11), int(8)), 3, 2)]
#cnn_win_sizes = [((int(8), int(6)), 2, 2)]
cnn_data = [lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, ssl_sat_samegrid, uvel_sat_samegrid, vvel_sat_samegrid]
# Get the cyclone and anti-cyclone rectangle 
cyclones_sat, anticyclones_sat = cnn.cnn_predict_grid(cnn_data, cnn_win_sizes, 0.96, model_fpath, storedir="sat1")

In [ ]:
# Group all predicted boxes according to overlap and number of rectangles in a group
cyc_sat, _ = cv2.groupRectangles(rectList=cyclones_sat, groupThreshold=1, eps=0.15)
acyc_sat, _ = cv2.groupRectangles(rectList=anticyclones_sat, groupThreshold=1, eps=0.15)
nCyc, nAcyc = len(cyc_sat), len(acyc_sat)

### Use labeled binary clusters for each cyclone or anti-cyclone based on Okubo Weiss parameter and vorticity to find appropriate box surrounding the eddy

In [ ]:
%%capture
# Create mask for OW values above threshold, one that contains cyclones (negative vorticity) 
# and vice versa for anti-cyclones
# Default Okubo Weiss value was -1, want to use -8 to be absolutely sure, we have a more loose threshold to
# indicate cells with vortex characteristics
OW_start = -0.1
OW,vorticity,OW_mask,OW_cyc_mask,OW_acyc_mask = calc_OW(lons_sinmod,lats_sinmod,uvel_sat_samegrid,vvel_sat_samegrid,OW_start)

# We label all unique mask clusters
OW_mask_labeled = measure.label(OW_mask)
cyc_mask_labeled_sat = measure.label(OW_cyc_mask)
acyc_mask_labeled_sat = measure.label(OW_acyc_mask)

# Make the predicted boxes encompass the full cyclone or anti-cyclone clusters and return info about the eddy
cyc_ctrIdxs, cyc_minOW, cyc_new_sat = tools.compare_tools.investigate_cluster(cyc_sat, OW, cyc_mask_labeled_sat, 'cyclone')
acyc_ctrIdxs, acyc_minOW, acyc_new_sat = tools.compare_tools.investigate_cluster(acyc_sat, OW, acyc_mask_labeled_sat, 'anti-cyclone')

# Eddy boxes given in lat-lon coordiantes
cyc_boxcoords_sat = box2coords(x_sinmod, y_sinmod, cyc_new_sat)
acyc_boxcoords_sat = box2coords(x_sinmod, y_sinmod, acyc_new_sat)

In [ ]:
x_sinmod

In [ ]:
##### import tools.compare_tools
importlib.reload(tools.compare_tools)
fig, ax = plt.subplots(3,2,figsize=(14,16), sharey=True, sharex=True)#, dpi=100)

cyc_box = box2coords(x_sinmod, y_sinmod, cyc_sat)
acyc_box = box2coords(x_sinmod, y_sinmod, acyc_sat)

ax[0,0].pcolormesh(x_sinmod, y_sinmod, OW_mask.T, cmap='nipy_spectral')
tools.compare_tools.plot_eddies(ax[0,0], cyc_box, 'r', numbered=False) 
tools.compare_tools.plot_eddies(ax[0,0], acyc_box, 'b', numbered=False)
ax[0,0].set_title('OW clusters', size=16)
ax[0,0].set_ylabel('y [km]', size=14)


test = np.zeros(OW_cyc_mask.shape)
test = np.where(OW_cyc_mask==1, 1, test)
test = np.where(OW_acyc_mask==1, 2, test)

ble = ax[0,1].pcolormesh(x_sinmod, y_sinmod, vorticity.T, cmap='bwr')
tools.compare_tools.plot_eddies(ax[0,1], cyc_box, 'r', numbered=False) 
tools.compare_tools.plot_eddies(ax[0,1], acyc_box, 'b', numbered=False) 
ax[0,1].set_title('Vorticity', size=16)

plt.colorbar(ble, ax=ax[0,1])

ax[1,0].pcolormesh(x_sinmod, y_sinmod, test.T, cmap='nipy_spectral')
tools.compare_tools.plot_eddies(ax[1,0], cyc_box, 'r', numbered=False) 
tools.compare_tools.plot_eddies(ax[1,0], acyc_box, 'b', numbered=False) 
ax[1,0].set_title('Labeling clusters', size=16)
ax[1,0].set_ylabel('y [km]', size=14)


ax[1,1].pcolormesh(x_sinmod, y_sinmod, cyc_mask_labeled_sat.T, cmap='nipy_spectral')
#ax[1,1].pcolormesh(x_sinmod, y_sinmod, acyc_mask_labeled_sat.T, cmap='nipy_spectral', alpha=0.5)
tools.compare_tools.plot_eddies(ax[1,1], cyc_box, 'r', numbered=False) 
#tools.compare_tools.plot_eddies(ax[1,1], acyc_boxcoords_sat, 'b', numbered=False) 
ax[1,1].set_title('Labeling clusters', size=16)

ax[2,0].pcolormesh(x_sinmod, y_sinmod, OW_mask_labeled.T, cmap='nipy_spectral')
tools.compare_tools.plot_eddies(ax[2,0], cyc_boxcoords_sat, 'r', numbered=False) 
tools.compare_tools.plot_eddies(ax[2,0], acyc_boxcoords_sat, 'b', numbered=False) 
ax[2,0].set_title('Include clusters', size=16)
ax[2,0].set_ylabel('y [km]', size=14)
ax[2,0].set_xlabel('x [km]', size=14)

ax[2,1].pcolormesh(x_sinmod, y_sinmod, ssl_sat_samegrid.T, cmap='rainbow')
ax[2,1].streamplot(x_sinmod, y_sinmod, uvel_sat_samegrid.T, vvel_sat_samegrid.T, density=5) 
tools.compare_tools.plot_eddies(ax[2,1], cyc_boxcoords_sat, 'r', numbered=False) 
tools.compare_tools.plot_eddies(ax[2,1], acyc_boxcoords_sat, 'b', numbered=False) 
ax[2,1].set_title('Final predictions', size=16)
ax[2,1].set_ylabel('y [km]', size=14)
ax[2,1].set_xlabel('x [km]', size=14)

plt.ylim([900000, 1200000])
plt.xlim([700000, 1200000])

### Show box surrounding the marked OW clusters before and after

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,12))
ax.imshow(OW_mask_labeled.T, cmap='nipy_spectral', origin='lower')
# Before
tools.compare_tools.plot_eddies(ax, [cyc_sat[0]], 'r', numbered=False) 
#tools.compare_tools.plot_eddies(ax, acyc_sat, 'b') 
# After
tools.compare_tools.plot_eddies(ax, [cyc_new_sat[0]], 'r', numbered=False) 
#tools.compare_tools.plot_eddies(ax, acyc_new_sat, 'b', numbered=False) 

ax.set_xlim([7, 25])
ax.set_ylim([4, 25])

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,12))
ax.imshow(vorticity.T, cmap='nipy_spectral', origin='lower')
# Before
plot_eddies(ax, cyc_sat, 'r') 
plot_eddies(ax, acyc_sat, 'b') 
# After
plot_eddies(ax, cyc_new_sat, 'r', numbered=False) 
plot_eddies(ax, acyc_new_sat, 'b', numbered=False) 

### Show plot of velocity streamplot and sea surface level with new boxes

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16,8), sharey=True)#, dpi=100)
ax[0].pcolormesh(x_sinmod, y_sinmod, ssl_sat_samegrid.T, cmap='rainbow')
ax[0].streamplot(x_sinmod, y_sinmod, uvel_sat_samegrid.T, vvel_sat_samegrid.T, density=8) 
cyc_box = box2coords(x_sinmod, y_sinmod, cyclones_sat)
acyc_box = box2coords(x_sinmod, y_sinmod, anticyclones_sat)
tools.compare_tools.plot_eddies(ax[0], cyc_box, 'r', numbered=False) 
tools.compare_tools.plot_eddies(ax[0], acyc_box, 'b', numbered=False) 
ax[0].set_ylim([900000, 1200000])
ax[0].set_xlim([700000, 1200000])
ax[0].set_ylabel('y [km]', size=14)
ax[0].set_xlabel('x [km]', size=14)
ax[0].set_title('Predictions with sliding window', size=16)

ax[1].pcolormesh(x_sinmod, y_sinmod, ssl_sat_samegrid.T, cmap='rainbow')
ax[1].streamplot(x_sinmod, y_sinmod, uvel_sat_samegrid.T, vvel_sat_samegrid.T, density=8) 
cyc_box = box2coords(x_sinmod, y_sinmod, cyc_sat)
acyc_box = box2coords(x_sinmod, y_sinmod, acyc_sat)
tools.compare_tools.plot_eddies(ax[1], cyc_box, 'r', numbered=False) 
tools.compare_tools.plot_eddies(ax[1], acyc_box, 'b', numbered=False) 
ax[1].set_ylim([900000, 1200000])
ax[1].set_xlim([700000, 1200000])
ax[1].set_xlabel('x [km]', size=14)
ax[1].set_title('Grouping predictions', size=16)




In [ ]:
# Eddy census dataframe to hold information about each eddy
cyc_census_df_sat = set_eddy_census(cyc_mask_labeled_sat, cyc_ctrIdxs, cyc_new_sat, cyc_minOW, lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, uvel_sat_samegrid, vvel_sat_samegrid, meastype='SINMOD')
acyc_census_df_sat = set_eddy_census(acyc_mask_labeled_sat, acyc_ctrIdxs, acyc_new_sat, acyc_minOW, lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, uvel_sat_samegrid, vvel_sat_samegrid, meastype='SINMOD')

# Process SINMOD data

### Perform pyramid sliding window on grid and return use CNN model to predict rectangles

In [ ]:
%%capture

model_fpath = 'D:/master/models/2016/cnn_mult_velocities_9652.h5'
cnn_win_sizes = [((int(8), int(6)), 2, 2),((int(11), int(8)), 3, 2),((int(14), int(11)), 3, 3)]
cnn_data = [lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, ssl_sinmod, uvel_sinmod, vvel_sinmod]
# Get the cyclone and anti-cyclone rectangle
cyclones_sinmod, anticyclones_sinmod = cnn.cnn_predict_grid(cnn_data, cnn_win_sizes, 0.95, model_fpath, storedir="sinmod1")

#import importlib
#import randomforest
#importlib.reload(randomforest)
#cyclones_sinmod, anticyclones_sinmod = randomforest.rf_predict_grid(cnn_data, cnn_win_sizes, 0.95, storedir="sinmod1")
#cyclones_sinmod, anticyclones_sinmod = svm.svm_predict_grid(cnn_data, cnn_win_sizes, 0.95, model_fpath, storedir="sinmod1")

In [ ]:
# Group all predicted boxes according to overlap and number of rectangles in a group
cyc_sinmod, _ = cv2.groupRectangles(rectList=cyclones_sinmod, groupThreshold=2, eps=0.15)
acyc_sinmod, _ = cv2.groupRectangles(rectList=anticyclones_sinmod, groupThreshold=2, eps=0.15)
nCyc, nAcyc = len(cyc_sinmod), len(acyc_sinmod)

### Use labeled binary clusters for each cyclone or anti-cyclone based on Okubo Weiss parameter and vorticity to find appropriate box surrounding the eddy

In [ ]:
%%capture
# Create mask for OW values above threshold, one that contains cyclones (negative vorticity) 
# and vice versa for anti-cyclones
# Default Okubo Weiss value was -1, want to use -8 to be absolutely sure, we have a more loose threshold to
# indicate cells with vortex characteristics
OW_start = -0.3
OW,vorticity,OW_mask,OW_cyc_mask,OW_acyc_mask = calc_OW(lons_sinmod,lats_sinmod,uvel_sinmod,vvel_sinmod,OW_start)

# We label all unique mask clusters
OW_mask_labeled = measure.label(OW_mask)
cyc_mask_labeled_sinmod = measure.label(OW_cyc_mask)
acyc_mask_labeled_sinmod = measure.label(OW_acyc_mask)

# Make the predicted boxes encompass the full cyclone or anti-cyclone clusters and return info about the eddy
cyc_ctrIdxs, cyc_minOW, cyc_new_sinmod = investigate_cluster(cyc_sinmod, OW, cyc_mask_labeled_sinmod, 'cyclone')
acyc_ctrIdxs, acyc_minOW, acyc_new_sinmod = investigate_cluster(acyc_sinmod, OW, acyc_mask_labeled_sinmod, 'anti-cyclone')

# Eddy boxes given in lat-lon coordiantes
cyc_boxcoords_sinmod = box2coords(x_sinmod, y_sinmod, cyc_new_sinmod)
acyc_boxcoords_sinmod = box2coords(x_sinmod, y_sinmod, acyc_new_sinmod)

### Show box surrounding the marked OW clusters before and after

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,12))
ax.imshow(OW_mask_labeled.T, cmap='nipy_spectral', origin='lower')
# Before
plot_eddies(ax, cyc_sinmod, 'r') 
plot_eddies(ax, acyc_sinmod, 'b') 
# After
plot_eddies(ax, cyc_new_sinmod, 'r', numbered=False) 
plot_eddies(ax, acyc_new_sinmod, 'b', numbered=False) 

### Show plot of velocity streamplot and sea surface level with new boxes

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,12))#, dpi=500)
ax.pcolormesh(x_sinmod, y_sinmod, ssl_sinmod.T, cmap='rainbow')
#ax.quiver(x_sinmod,y_sinmod,uvel_sinmod.T,vvel_sinmod.T,scale=5)
ax.streamplot(x_sinmod, y_sinmod, uvel_sinmod.T, vvel_sinmod.T, density=10) 
plot_eddies(ax, cyc_boxcoords_sinmod, 'r') 
plot_eddies(ax, acyc_boxcoords_sinmod, 'b') 

In [ ]:
cyc_census_df_sinmod = test.set_eddy_census(OW, cyc_mask_labeled_sinmod, cyc_ctrIdxs, cyc_new_sinmod, cyc_minOW, lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, uvel_sinmod, vvel_sinmod, meastype='SINMOD')
acyc_census_df_sinmod = test.set_eddy_census(OW, acyc_mask_labeled_sinmod, acyc_ctrIdxs, acyc_new_sinmod, acyc_minOW, lons_sinmod, lats_sinmod, x_sinmod, y_sinmod, uvel_sinmod, vvel_sinmod, meastype='SINMOD')

# Compare the two

### Eddy census

In [ ]:
cyc_census_df_sat

In [ ]:
acyc_census_df_sat

In [ ]:
cyc_census_df_sinmod

In [ ]:
acyc_census_df_sinmod

### Compare data; temperature, sea surface level, streamplot

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(18,18))

def plot_eddies_wrapper(ax, cyc, acyc):
    plot_eddies(ax, cyc, 'r') 
    plot_eddies(ax, acyc, 'b') 

axes[0,0].contourf(x_sinmod, y_sinmod, sst_sat_samegrid.T, levels=40, cmap='rainbow')#, vmin=6.5, vmax=10.5)
plot_eddies_wrapper(fig, axes[0,0], cyc_boxcoords_sat, acyc_boxcoords_sat)
axes[0,0].set_title('SST Satelite')

axes[0,1].contourf(x_sinmod, y_sinmod, sst_sinmod.T, levels=40, cmap='rainbow')
plot_eddies_wrapper(axes[0,1], cyc_boxcoords_sinmod, acyc_boxcoords_sinmod)
axes[0,1].set_title('SST SINMOD')

axes[1,0].contourf(x_sinmod, y_sinmod, ssl_sat_samegrid.T, levels=40, cmap='rainbow')
plot_eddies_wrapper(axes[1,0], cyc_boxcoords_sat, acyc_boxcoords_sat)
axes[1,0].set_title('SSL Satelite')

axes[1,1].contourf(x_sinmod, y_sinmod, ssl_sinmod.T, levels=40, cmap='rainbow')
plot_eddies_wrapper(axes[1,1], cyc_boxcoords_sinmod, acyc_boxcoords_sinmod)
axes[1,1].set_title('SSL SINMOD')

axes[2,0].contourf(x_sinmod, y_sinmod, sal_sat_samegrid.T, levels=80, cmap='rainbow', vmin=33, vmax=35.6)
plot_eddies_wrapper(axes[2,0], cyc_boxcoords_sat, acyc_boxcoords_sat)
axes[2,0].set_title('Salinity Satelite')

axes[2,1].contourf(x_sinmod, y_sinmod, sal_sinmod.T, levels=100, cmap='rainbow', vmin=33, vmax=35.6)
plot_eddies_wrapper(axes[2,1], cyc_boxcoords_sinmod, acyc_boxcoords_sinmod)
axes[2,1].set_title('Salinity SINMOD')

In [ ]:
import datetime as dt
origin = dt.date(1950, 1, 1)
print(origin + dt.timedelta(hours=582192))
print(origin + dt.timedelta(hours=582192))
print(origin + dt.timedelta(hours=582360))
print(origin + dt.timedelta(hours=582528))
print(origin + dt.timedelta(hours=582696))


In [ ]:
import numpy as np
t = np.array([[1,1],[1,0]])
x = np.array([[1,1],[1,1]])
y = np.array([[1,1],[1,1]])

idx = np.where(t==1)
print(idx)
print(len(idx[0]))
np.sum(np.sqrt(np.square(x[idx]) + np.square(y[idx])))/3